In [1]:
import os, glob
import sys
import json
from PIL import Image
from collections import Counter

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tifffile as tiff
import matplotlib.pyplot as plt
from tqdm import tqdm

import cv2

from sklearn.model_selection import KFold

sys.path.append("detection-wheel")

In [2]:
EPOCHS = 60

In [3]:
import os
import numpy as np
import torch
from PIL import Image
from torchvision.transforms import functional as F, InterpolationMode

class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, imgs, masks, transforms, mode='train'):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = imgs#sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/image/*.png'))
        self.masks = masks#sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/mask/*.png'))
        self.mode = mode

    def __getitem__(self, idx):
        # load images and masks
        img_path = self.imgs[idx]
        mask_path = self.masks[idx]
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path).convert('L')
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        #masks = (mask == obj_ids[:, None, None])
        #print((obj_ids[:, None, None]).shape)
        #masks = mask == obj_ids[:, None, None]
        masks = [np.where(mask== obj_ids[i, None, None],1,0) for i in range(len(obj_ids))]
        masks = np.array(masks)

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.nonzero(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        try:
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            #print(area,area.shape,area.dtype)
        except:
            area = torch.tensor([[0],[0]])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        #print(masks.shape)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

            if self.mode == 'train':
                rand_width_height = 512 # np.random.randint(512, 1024)
                # resize to 1024x1024
                new_width = rand_width_height
                new_height = rand_width_height
                orig_height, orig_width = img.shape[1:]
                # orig_height = img.size[1]
                # orig_width = img.size[0]
                
                img = F.resize(img, [new_height, new_width], interpolation=InterpolationMode.BILINEAR)

                if target is not None:
                    target["boxes"][:, 0::2] *= new_width / orig_width
                    target["boxes"][:, 1::2] *= new_height / orig_height
                    if "masks" in target:
                        target["masks"] = F.resize(
                            target["masks"], [new_height, new_width], interpolation=InterpolationMode.NEAREST
                        )


        return img, target

    def __len__(self):
        return len(self.imgs)

In [4]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.RandomIoUCrop())
        transforms.append(T.RandomZoomOut())
        transforms.append(T.RandomPhotometricDistort())
        transforms.append(T.ScaleJitter())
        transforms.append(T.RandomShortestSize())
        # transforms.append(T.FixedSizeCrop(size=(512,512)))
        
        
    return T.Compose(transforms)

In [5]:
from engine import train_one_epoch, evaluate
import utils

In [6]:
n_imgs = len(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/image/*'))
n_imgs



1622

In [7]:
from sklearn.model_selection import train_test_split

all_imgs = sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/image/*.png'))
all_masks = sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/mask/*.png'))
all_imgs = np.array(all_imgs)
all_masks = np.array(all_masks)


indices = [i for i in range(len(all_imgs))]
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=1234)

train_img = all_imgs[train_indices]
train_mask = all_masks[train_indices]
val_img = all_imgs[val_indices]
val_mask = all_masks[val_indices]
dataset_train = PennFudanDataset(train_img, train_mask, get_transform(train=True), mode='train')
dataset_val = PennFudanDataset(val_img, val_mask, get_transform(train=False), mode='val')
train_dl = torch.utils.data.DataLoader(
    dataset_train, batch_size=1, shuffle=True, num_workers=os.cpu_count(), pin_memory=True, drop_last=True, collate_fn=utils.collate_fn)
val_dl = torch.utils.data.DataLoader(
    dataset_val, batch_size=1, shuffle=False, num_workers=os.cpu_count(), pin_memory=True,collate_fn=utils.collate_fn)


In [8]:
img, target = train_dl.dataset[0]
# img.shape, target['masks']
target['masks'].shape, img.shape, img.min(), img.max() # -> (torch.Size([8, 512, 512]), torch.Size([3, 512, 512]), tensor(0.), tensor(1.))


/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(torch.Size([4, 512, 512]),
 torch.Size([3, 512, 512]),
 tensor(0.0128),
 tensor(0.6728))

# Create train dataset which is an augmentation of the original train dataset

In [9]:
# # load /home/viktor/Documents/kaggle/hubmap-2023/experiments/yolo/yolov8-vanilla/YOLOv8x-maskrcnn-augms/datasets/train/images/0a4ddecc55f0.tif
# img = tiff.imread('/home/viktor/Documents/kaggle/hubmap-2023/experiments/yolo/yolov8-vanilla/YOLOv8x-maskrcnn-augms/datasets/train/images/0a4ddecc55f0.tif')
# img.min(), img.max(), img.shape # -> (0, 255, (512, 512, 3))

In [10]:
mask = target['masks'][0]
mask.shape # -> torch.Size([512, 512])

torch.Size([512, 512])

In [11]:
import cv2
import numpy as np

# Assuming mask is a 2D torch tensor of shape [H, W]
mask_np = mask.cpu().numpy() # Convert the mask to a numpy array

# Find contours. Note, this will give the boundary pixels, 
# so polygons will be represented as a list of points along the boundary
contours, _ = cv2.findContours(mask_np.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

polygons = [cnt.reshape(-1, 2) for cnt in contours] # Reshape for easier handling

# Each element in polygons now represent a separate connected component in the original mask
# polygons # [array([[121,  10], [120,  11], [119,  11],

In [12]:
folder_root = "datasets/train-2/"

In [13]:
img.shape

torch.Size([3, 512, 512])

In [14]:
import torch
from PIL import Image

# Assuming that torch_tensor is your image tensor
torch_tensor = img

# Denormalize the tensor from [0,1] to [0,255]
torch_tensor_denorm = (torch_tensor * 255).byte()

# Permute the tensor to make it suitable for creating a PIL image
img_pil = Image.fromarray(torch_tensor_denorm.cpu().numpy().transpose(1,2,0))

# Save the image
img_pil.save('image.tif')


In [15]:
from tqdm import tqdm

n_epochs = 16

train_dl_indx = 0

for epoch in tqdm(range(n_epochs)):
    for images, targets in train_dl:
        
        img = images[0]
        target = targets[0]
        masks = target['masks']
        
        
        # save img 
        # Assuming that torch_tensor is your image tensor
        torch_tensor = img

        # Denormalize the tensor from [0,1] to [0,255]
        torch_tensor_denorm = (torch_tensor * 255).byte()

        # Permute the tensor to make it suitable for creating a PIL image
        img_pil = Image.fromarray(torch_tensor_denorm.cpu().numpy().transpose(1,2,0))

        # Save the image
        img_pil.save(f'datasets/train/images/{train_dl_indx}.tif')
        

        # save the mask
        
        label_txt = ''
        
        
        for mask in masks:
        
        
            # get polygons
            # Assuming mask is a 2D torch tensor of shape [H, W]
            mask_np = mask.cpu().numpy() # Convert the mask to a numpy array

            # Find contours. Note, this will give the boundary pixels, 
            # so polygons will be represented as a list of points along the boundary
            contours, _ = cv2.findContours(mask_np.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            coordinates = [cnt.reshape(-1, 2) for cnt in contours] # Reshape for easier handling
            
            
            label_txt += '0 '
            # Normalize
            coor_array = np.array(coordinates[0]).astype(float)
            coor_array /= float(512)
            # transform to str
            coor_list = list(coor_array.reshape(-1).astype(str))
            coor_str = ' '.join(coor_list)
            # add string to label txt
            label_txt += f'{coor_str}\n'
            
        # Write labels to txt file
        with open(f'datasets/train/labels/{train_dl_indx}.txt', 'w') as f:
            f.write(label_txt)
        
        train_dl_indx += 1

100%|██████████| 16/16 [09:26<00:00, 35.40s/it]
